The objective of this notebook is test the ability to learn a side problem, like finding nearest neighbour

We start creataing the data

In [1]:
from math import sqrt
import torch
from torch_geometric.data import Data
from random import randint
from sys import float_info

In [2]:
instances = {}
for k in range(0, 5000):
    nodes = {}
    for i in range(0, 50):
        lat_i = randint(0, 100)
        lon_i = randint(0, 100)
        node_i = (lat_i, lon_i)
        lat_j = randint(0, 100)
        lon_j = randint(0, 100)
        node_j = (lat_j, lon_j)
        nodes[i + 1] = node_i
        nodes[i + 51] = node_j

    dist = {}
    pairs = {}
    for i in range(1, 101):
        for j in range(1, 101):
            if i != j:
                dist[i,j] = sqrt( (nodes[i][0] - nodes[j][0])**2 + (nodes[i][1] - nodes[j][1])**2 )
            else:
                dist[i,j] = float_info.max
    for i in range(1, 101):
        for j in range(1, 101):
            if i not in pairs:
                pairs[i] = j
            if i != j:
                if dist[i,j] < dist[i,pairs[i]]:
                    pairs[i] = j

    nodes[0] = (0,0)
    for i in range(1,101):
        dist[0,i] = sqrt( (nodes[0][0] - nodes[i][0])**2 + (nodes[0][1] - nodes[i][1])**2 )
        dist[i,0] = dist[0,i]
    y = [[0 for _ in range(101)] for _ in range(101)]
    for i in range(101):
        if i > 0:
            y[i][pairs[i]] = 1
                
    instances[k] = {"nodes": nodes, "dist": dist, "y": y}

Experiment 1. Use Torch Geometric model based on message passing edge convolution

In [3]:

import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

In [4]:
complete_graph_list = []
for i in range(101):
    for j in range(101):
        if i != j:
            complete_graph_list.append([i,j])
edge_index = torch.tensor(complete_graph_list, dtype=torch.double).t().contiguous()
n_edges = len(complete_graph_list)

In [5]:
data_list = []
for instance_name in instances:
    y = torch.tensor(instances[instance_name]["y"], dtype=torch.double)
    x = torch.tensor([instances[instance_name]["nodes"][i] for i in range(0, 101)], dtype=torch.double)
    attr = [[i] for i in range(n_edges)]
    cnt = -1
    for i in range(101):
        tmp_row = []
        for j in range(101):
            if i != j:
                # cnt += 1
                # attr[cnt].append(instances[instance_name]["dist"][i,j])
                tmp_row.append(instances[instance_name]["dist"][i,j])
        min_row = min(tmp_row)
        max_row = max(tmp_row)
        tiny_cnt = 0
        for j in range(101):
            if i != j:
                cnt += 1
                attr[cnt].append((tmp_row[tiny_cnt] - min_row)/(max_row - min_row))
                tiny_cnt += 1
    attr = torch.tensor(attr, dtype=torch.double)
    pos = []
    for i in range(101):
        pos.append(instances[instance_name]["nodes"][i])
    pos = torch.tensor(pos, dtype=torch.double)
    # ## filtering by TW, strict
    # complete_graph_list = []
    # for i in range(101):
    #     for j in range(101):
    #         if i!=j:
    #             try:
    #                 if instance_dict[instance_name][i][5] + instance_dict[instance_name][i][6] + loc_dict[i][j] < instance_dict[instance_name][i][5]:
    #                     complete_graph_list.append([i,j])
    #             except:
    #                 continue
    # edge_index = torch.tensor(complete_graph_list, dtype=torch.double).t().contiguous()
    ## end filtering
    data_list.append(Data(x=x, y=y, edge_index=edge_index, pos=pos, edge_attr=attr))

In [6]:
# data_source = Instances(data_list)
dataloader = DataLoader(data_list[0:1000], batch_size=1)
data_test = DataLoader(data_list[1000:1200], batch_size=1)
# datatorch = data_source.to_conv_nets(start=428, end=1458, batch_size=10)
# torchtest = data_source.to_conv_nets(start=1458, end=488, batch_size=10)

Ex1 model starts here

In [41]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU, Softmax
from torch_geometric.nn import MessagePassing

class EdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='min') #  "Max" aggregation.
        self.mlp = Seq(Linear(2 * in_channels, out_channels),
                       ReLU(),
                       Linear(out_channels, out_channels),
                       Softmax(dim=1))

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # x_i has shape [E, in_channels]
        # x_j has shape [E, in_channels]

        tmp = torch.cat([x_i, x_j], dim=1)  # tmp has shape [E, 2 * in_channels]
        return self.mlp(tmp)

In [33]:
from torch_geometric.nn import knn_graph

class DynamicEdgeConv(EdgeConv):
    def __init__(self, in_channels, out_channels, k=6):
        super().__init__(in_channels, out_channels)
        self.k = k
        self.double()

    def forward(self, x, batch=None):
        edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow)
        return super().forward(x, edge_index)

NameError: name 'EdgeConv' is not defined

ONLY KNN

In [62]:
from torch_geometric.nn import knn_graph, GraphConv

class DynamicEdgeConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DynamicEdgeConv, self).__init__()
        self.k = 2
        self.conv1 = GraphConv(in_channels, out_channels)
        self.double()

    def forward(self, x, batch=None):
        # x = self.conv1(x, edge_index, None).relu()
        edge_index = knn_graph(x, self.k, batch, loop=False)
        
        return edge_index

In [8]:
def myCustomLoss(my_outputs, objective_matrix):
    #specifying the batch size
    my_batch_size = my_outputs.size()[0]
    #selecting the values that correspond to labels
    diff = torch.abs(my_outputs - objective_matrix)
    return torch.sum(diff*diff)/(101*101)

In [85]:
def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()
        # print(data.x, data.edge_index, data.batch)
        output = model(data.x)
        output = output.tolist()
        # print(output, data.y)
        # print(output)
        matrix_output = [[0 for _ in range(101)] for _ in range(101)]
        for i in range(len(output[0])):
            matrix_output[int(output[0][i])][int(output[1][i])] = 1
        output = torch.tensor(matrix_output, dtype=torch.double)
        # print(torch.sum(output))
        loss = myCustomLoss(output, data.y)
        loss.requires_grad = True
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
        test_size = 101*101*1 #extract batch size
    return loss_all / test_size

def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x)
        output = output.tolist()
        # pred = torch.tensor([[1 if j>0.5 else 0 for j in i] for i in output.tolist()], dtype=torch.double)
        matrix_output = [[0 for _ in range(101)] for _ in range(101)]
        for i in range(len(output[0])):
            matrix_output[int(output[0][i])][int(output[1][i])] = 1
        pred = torch.tensor(matrix_output, dtype=torch.double)
        correct += pred.eq(data.y).sum().item()
        test_size = 101*101*1 #extract batch size
    return correct / test_size / len(loader)

In [10]:
from copy import deepcopy

In [60]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DynamicEdgeConv(2, 101).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model_dict = {}

for epoch in range(1, 5):
    loss = train(epoch)
    train_acc = test(dataloader)
    test_acc = test(data_test)
    model_dict[epoch] = {"model": deepcopy(model), "loss": loss, "trainAcc": train_acc, "testAcc": test_acc}
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

UnboundLocalError: cannot access local variable 'edge_index' where it is not associated with a value

In [86]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DynamicEdgeConv(2,101).to(device)
optimizer = torch.optim.Adam(model.parameters())
model_dict = {}

for epoch in range(1, 5):
    loss = train(epoch)
    train_acc = test(dataloader)
    test_acc = test(data_test)
    model_dict[epoch] = {"model": deepcopy(model), "loss": loss, "trainAcc": train_acc, "testAcc": test_acc}
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 0.0013, Train Acc: 0.9871, Test Acc: 0.9871
Epoch: 002, Loss: 0.0013, Train Acc: 0.9871, Test Acc: 0.9871
Epoch: 003, Loss: 0.0013, Train Acc: 0.9871, Test Acc: 0.9871
Epoch: 004, Loss: 0.0013, Train Acc: 0.9871, Test Acc: 0.9871


Evaluation

In [89]:
best_model = model_dict[max(model_dict, key=lambda k: model_dict[k]["testAcc"])]["model"]
best_model.eval()
data_show = data_list[430]
pred = best_model(data_show.x).tolist()
pred = model(data_show.x).tolist()

In [88]:
print( data_list[430])

Data(x=[101, 2], edge_index=[2, 10100], edge_attr=[10100, 2], y=[101, 101], pos=[101, 2])


In [90]:
matrix_output = [[0 for _ in range(101)] for _ in range(101)]
for i in range(len(pred[0])):
    matrix_output[int(pred[0][i])][int(pred[1][i])] = 1
pred = torch.tensor(matrix_output, dtype=torch.double).tolist()
for row in pred:
    print(row, sum(row))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [91]:
edge_index = knn_graph(data_show.x, 3)

In [92]:
print(edge_index)

tensor([[ 64,  50,   1,  77,  35,  64,  97,  79,  96,  39, 100,  99,  36,  76,
          70,  83,  80,  33,  46,  55,  24,  33,  82,  89,  89,  16,   7,  35,
          77,  81,  29,  47,  80,  37,  38,  82,  31,  76,  70,  19,  30,  90,
          71,  79,  75,  28,  62,  60,  99,   8,  98,  86,  91,  27,  45,  74,
          51,  65,  90,  13,  63,  59,  61,  99, 100,  98,  25,  69,  30,  27,
         100,  39,  55,  52,  87,  22,  69,  30,  75,  87,  44,  91,  39,  41,
          15,  62,  60,  10,  47,  80,  25,  22,  67,  48,  32,  12,  48,  31,
          73,   7,  93,  83,  98,  58,  82,  77,   9,  81,   4,  76,  70,  11,
           5,  73,  82,  58,  11,  41,   3,  27,  42,  68,  65,  39,  43,  27,
          40,  65,  28,  53,  41,  39,  92,  26,  14,  18,  74,  67,   6,  40,
          42,  29,  80,  10,  32,  31,  36,  78,  95,   6,  78,  64,  95,  18,
          53,  85,  87,  72,  26,  43,  41,  57,  94,  85,  51,  24,  52,   6,
          60,  62,  10,  63,  59,  20,  38,  34,  82

In [93]:
matrix_output = [[0 for _ in range(101)] for _ in range(101)]
for i in range(len(edge_index[0])):
    matrix_output[int(edge_index[0][i])][int(edge_index[1][i])] = 1
pred = torch.tensor(matrix_output, dtype=torch.double).tolist()
for row in pred:
    print(row, sum(row))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0.0
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [69]:
max_val = -2
min_val = 2
vals = []
for i in pred:
    vals += i
    if max(i) > max_val:
        max_val = max(i)
    if min(i) < min_val:
        min_val = min(i)
from statistics import mean, stdev
avg_val = mean(vals)
sd_val = stdev(vals)
threshold = (max_val + min_val)/2

TypeError: can't convert type 'Tensor' to numerator/denominator

In [70]:
row_dif_dict = {}
for k in range(0, 1000):
    row_dif = 0
    for i in range(len(pred)):
        # print(sum([1 if j > avg_val + 3.9 * sd_val else 0 for j in pred[i]]), int(sum(data_show.y[i])), abs(sum([1 if j > avg_val + 3.9 * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i]))))
        row_dif += abs(sum([1 if j > avg_val + (0 + k * 0.1) * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i])))
    row_dif_dict[k] = row_dif
best_threshold = min(row_dif_dict, key=lambda k: row_dif_dict[k])
# for i in range(len(pred)):
#     print(sum([1 if j > avg_val + (3.9 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]]), int(sum(data_show.y[i])), abs(sum([1 if j > avg_val + (3.9 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i]))))
# print(best_threshold, row_dif_dict[best_threshold])
for i in range(len(pred)):
    print([1 if j > avg_val + (0 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]], sum([1 if j > avg_val + (0 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]]))
cnt = 0
for i in range(len(pred)):
    cnt += sum([1 if j > avg_val + (0 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]])
    if cnt > 0:
        print(i)
print(cnt)

KeyboardInterrupt: 

In [38]:
for i in range(len(pred)):
    print(data_show.y[i].tolist(), sum(data_show.y[i].tolist()))
print(sum(sum(y)))
# print(data_show.y[49].tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

Model 2. Graph Convolution

In [50]:
from torch_geometric.nn import GraphConv, global_add_pool, Linear
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_features = 2
        dim = 101*101

        self.conv1 = GraphConv(num_features, dim)
        self.conv2 = GraphConv(dim, dim)

        self.lin1 = Linear(dim, dim)

        self.double()

    def forward(self, x, edge_index, batch, edge_weight=None):
        # x, edge_index = data.x, data.edge_index

        # x = self.conv1(x, edge_index)
        # x = F.relu(x)
        # x = F.sigmoid(self.conv2(x, edge_index))
        # x = F.dropout(x, training=self.training)
        # x = self.conv2(x, edge_index)

        x = self.conv1(x, edge_index, edge_weight).relu()
        x = self.conv2(x, edge_index, edge_weight).relu()
        x = global_add_pool(x, batch)
        x = self.lin1(x).relu()
        # x = F.dropout(x, p=0.5, training=self.training)
        # x = self.lin2(x)
        # x = torch.reshape(x, (101, 101))
        return x

In [75]:
def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()
        # print(data.x, data.edge_index, data.batch)
        output = model(data.x, data.edge_index.long(), data.batch)[0]
        # print(output, data.y)
        # print(output)
        # print(len(output[0]))
        # print(data.y.tolist())
        obj_list = data.y.tolist()
        obj = []
        for i in range(101):
            for j in range(101):
                # if i != j:
                    obj.append(obj_list[i][j])
        obj = torch.tensor(obj, dtype=torch.float32)
        # print(len(data.y.tolist()), print(len(output[0].tolist())))
        loss = F.cross_entropy(output, obj, reduction='mean')
        loss.sum().backward()
        loss_all += loss.sum().item() * data.num_graphs
        optimizer.step()
        test_size = 101*101*1 #extract batch size
    return loss_all / test_size

def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.edge_index.long(), data.batch)[0]
        # op_list = output.tolist()
        obj_list = data.y.tolist()
        obj = []
        for i in range(101):
            for j in range(101):
                # if i != j:
                    obj.append(obj_list[i][j])
        obj = torch.tensor(obj, dtype=torch.float32)
        # pred = torch.tensor([[1.0 if op_list[i][j]>0.5 else 0 for j in range(101)] for i in range(101)], dtype=torch.double)
        correct += output.eq(obj).sum().item()
        test_size = 101*101*1 #extract batch size
    return correct / test_size / len(loader)

In [76]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model_dict = {}

for epoch in range(1, 3):
    loss = train(epoch)
    train_acc = test(dataloader)
    test_acc = test(data_test)
    model_dict[epoch] = {"model": deepcopy(model), "loss": loss, "trainAcc": train_acc, "testAcc": test_acc}
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 197629452.9485, Train Acc: 0.8034, Test Acc: 0.7925
Epoch: 002, Loss: 11994.1087, Train Acc: 0.8163, Test Acc: 0.8211


In [87]:
best_model = model_dict[max(model_dict, key=lambda k: model_dict[k]["testAcc"])]["model"]
best_model.eval()
data_show = data_list[430]
pred = best_model(data_show.x, data_show.edge_index.long(), data_show.batch).reshape(101,101).tolist()

In [94]:
rows = []
for i in pred:
    tmp = []
    for j in i:
        if j > 0.01:
            tmp.append(1)
        else:
            tmp.append(0)
    rows.append(tmp)
for i in rows:
    print(i, sum(i))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] 9
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0] 13
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [78]:
max_val = -2
min_val = 2
vals = []
for i in pred:
    vals += i
    if max(i) > max_val:
        max_val = max(i)
    if min(i) < min_val:
        min_val = min(i)
from statistics import mean, stdev
avg_val = mean(vals)
sd_val = stdev(vals)
threshold = (max_val + min_val)/2

In [79]:
row_dif_dict = {}
for k in range(0, 1000):
    row_dif = 0
    for i in range(len(pred)):
        # print(sum([1 if j > avg_val + 3.9 * sd_val else 0 for j in pred[i]]), int(sum(data_show.y[i])), abs(sum([1 if j > avg_val + 3.9 * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i]))))
        row_dif += abs(sum([1 if j > avg_val + (0 + k * 0.1) * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i])))
    row_dif_dict[k] = row_dif
best_threshold = min(row_dif_dict, key=lambda k: row_dif_dict[k])
# for i in range(len(pred)):
#     print(sum([1 if j > avg_val + (3.9 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]]), int(sum(data_show.y[i])), abs(sum([1 if j > avg_val + (3.9 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i]))))
# print(best_threshold, row_dif_dict[best_threshold])
for i in range(len(pred)):
    print([1 if j > avg_val + (0 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]], sum([1 if j > avg_val + (0 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]]))
cnt = 0
for i in range(len(pred)):
    cnt += sum([1 if j > avg_val + (0 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]])
    if cnt > 0:
        print(i)
print(cnt)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [80]:
for i in data_show.y.tolist():
    print(i, sum(i))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

New simple model

The plan is to simply consider the problem of a point as a separation problem between neighbors as clustering per distance. Let's consider a problem with d = (i = (|V| - 1)) + (l = 2) + (t = 2) dimensions. Dimensions i represent the remainder nodes distances, l represent the current position and t the time windows. Each node has this properties. This simple net will have at least 3 neurons with no depth.

In [66]:
import numpy as np
import keras as ks
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

In [103]:
X = np.matrix([[instances[k]["dist"][i,j] if i != j else 0.0 for j in range(101)] for k in range(1000) for i in range(101)])
y = np.matrix([[1 if instances[k]["y"][i][j] > 0.5 else 0 for j in range(101)] for k in range(1000) for i in range(101)])

In [105]:
loss_fun = ks.losses.BinaryFocalCrossentropy(alpha=0.99, gamma=3.0, from_logits=False)
model1 = Sequential()
model1.add(Dense(202, input_dim=101, activation='relu'))
model1.add(Dense(101, activation='sigmoid'))
# model2 = Sequential()
# model2.add(Dense(5, input_dim=101, activation='relu'))
# model2.add(Dense(1, activation='sigmoid'))
# model3 = Sequential()
# model3.add(Dense(20, input_dim=101, activation='relu'))
# model3.add(Dense(1, activation='sigmoid'))
# compile the models. Here we need to chose an optimiser. This one is called adam, it is used to
# determine how the training is performed. We do not care in this lecture how this is done.
opt = ks.optimizers.Adam(learning_rate=0.02)
model1.compile(loss=loss_fun, optimizer=opt, metrics=['accuracy'])
# model2.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
# model3.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
#fit the models on the dataset
model1.fit(X, y, epochs=10, batch_size=5, verbose = True)
# model2.fit(X, y, epochs=30, batch_size=5, verbose = False)
# model3.fit(X, y, epochs=30, batch_size=5, verbose = False)
# evaluate the keras model
print('Model 1:')
_, accuracy1 = model1.evaluate(X, y)
# print('Model 2:')
# _, accuracy2 = model2.evaluate(X, y)
# print('Model 3:')
# _, accuracy3 = model3.evaluate(X, y)

Epoch 1/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step - accuracy: 0.0100 - loss: 0.0436
Epoch 2/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 24s 1ms/step - accuracy: 0.0100 - loss: 0.0098
Epoch 3/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 22s 1ms/step - accuracy: 0.0104 - loss: 0.0098
Epoch 4/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 22s 1ms/step - accuracy: 0.0094 - loss: 0.0098
Epoch 5/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.0101 - loss: 0.0098
Epoch 6/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.0101 - loss: 0.0098
Epoch 7/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.0103 - loss: 0.0098
Epoch 8/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.0101 - loss: 0.0098
Epoch 9/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.0095 - loss: 0.0098
Epoch 10/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.0102 - loss: 0.0098
Model 1:
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 3s 891us/step - accuracy: 0.0099 - loss: 0.0099


In [106]:
for i in range(101):
    op = [instances[1]["y"][i][j] for j in range(101)]
    print(i, sum(op), op)

0 0 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1 1 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2 1 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3 1 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [107]:
Z1 = model1.predict(np.matrix([instances[1]["dist"][1,j] if 1 != j else sqrt(100*100.0 + 100*100.0) for j in range(101)]))
# Z2 = model2.predict(np.c_[xx.ravel(), yy.ravel()])
# Z2 = Z2.reshape(xx.shape)
# Z3 = model3.predict(np.c_[xx.ravel(), yy.ravel()])
# Z3 = Z3.reshape(xx.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


In [108]:
from statistics import mean, stdev

In [109]:
print(Z1)
print(sum(sum(Z1)))
print(max(Z1[0]), min(Z1[0]), mean(Z1[0]), stdev(Z1[0]))

[[0.00361545 0.21006188 0.2011656  0.23723958 0.2073307  0.21382491
  0.22490281 0.22450343 0.22132063 0.1744751  0.2571711  0.2207405
  0.23040491 0.19213878 0.24252059 0.20949587 0.23388769 0.2037007
  0.23714536 0.21551818 0.20802212 0.22077312 0.20651841 0.21837643
  0.20268373 0.20499697 0.18135248 0.22214161 0.24821514 0.23772684
  0.24078953 0.22750582 0.21247354 0.22886954 0.2262112  0.23306741
  0.17926775 0.20146549 0.21807727 0.22241148 0.221768   0.24072385
  0.23905748 0.20799132 0.22482333 0.22680211 0.21305695 0.2060667
  0.24007073 0.19606557 0.2101577  0.18854213 0.19042547 0.2214805
  0.22004431 0.21263133 0.23416989 0.18797512 0.2158055  0.25655043
  0.20733106 0.22743937 0.2097033  0.1706956  0.17988807 0.20124602
  0.21509963 0.20844914 0.19446303 0.24361078 0.22078308 0.23665059
  0.28257138 0.27594218 0.2101993  0.23032126 0.24495094 0.20286645
  0.21808533 0.24996866 0.23834963 0.18276879 0.19420427 0.22036037
  0.17119575 0.21922007 0.17248552 0.19776706 0.1959

In [110]:
print(np.matrix([float(instances[1]["y"][1][j]) for j in range(101)]))
print(sum([(instances[1]["y"][1][j]) for j in range(101)]))
print(instances[1]["y"][1][46], Z1.tolist()[0][46])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]
1
1 0.21305695176124573


In [31]:
print(sum(sum((Z1-np.matrix([float(instances[1]["y"][1][j]) for j in range(101)])[0].tolist()))))

14.119866843189811


A second Keras model. Tends to work relatively good or at least makes bad guesses with a great sense of security. Activation linear

In [111]:
loss_fun = ks.losses.BinaryFocalCrossentropy(alpha=0.99, gamma=3.0, from_logits=False)
model1_1 = Sequential()
model1_1.add(Dense(101, input_dim=101, activation='linear'))
model1_1.add(Dense(101, activation='sigmoid'))
# model2 = Sequential()
# model2.add(Dense(5, input_dim=101, activation='relu'))
# model2.add(Dense(1, activation='sigmoid'))
# model3 = Sequential()
# model3.add(Dense(20, input_dim=101, activation='relu'))
# model3.add(Dense(1, activation='sigmoid'))
# compile the models. Here we need to chose an optimiser. This one is called adam, it is used to
# determine how the training is performed. We do not care in this lecture how this is done.
opt = ks.optimizers.Adam(learning_rate=0.02)
model1_1.compile(loss=loss_fun, optimizer=opt, metrics=['accuracy'])
# model2.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
# model3.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
#fit the models on the dataset
model1_1.fit(X, y, epochs=10, batch_size=5, verbose = True)
# model2.fit(X, y, epochs=30, batch_size=5, verbose = False)
# model3.fit(X, y, epochs=30, batch_size=5, verbose = False)
# evaluate the keras model
print('Model 1:')
_, accuracy1 = model1_1.evaluate(X, y)
# print('Model 2:')
# _, accuracy2 = model2.evaluate(X, y)
# print('Model 3:')
# _, accuracy3 = model3.evaluate(X, y)

Epoch 1/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 14s 648us/step - accuracy: 0.0201 - loss: 6.1355
Epoch 2/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 18s 913us/step - accuracy: 0.0234 - loss: 6.5438
Epoch 3/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 18s 903us/step - accuracy: 0.0236 - loss: 6.6278
Epoch 4/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 924us/step - accuracy: 0.0230 - loss: 6.4816
Epoch 5/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 18s 909us/step - accuracy: 0.0229 - loss: 6.6570
Epoch 6/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 18s 913us/step - accuracy: 0.0230 - loss: 6.5277
Epoch 7/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 931us/step - accuracy: 0.0238 - loss: 6.4288
Epoch 8/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 942us/step - accuracy: 0.0237 - loss: 6.5622
Epoch 9/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 922us/step - accuracy: 0.0232 - loss: 6.6897
Epoch 10/10
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 924us/step - accuracy: 0.0224 - loss: 6.5072
Model 1:
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 3s 841us/step - accuracy: 0.0

In [112]:
Z1 = model1_1.predict(np.matrix([instances[1]["dist"][3,j] if 3 != j else sqrt(100*100.0 + 100*100.0) for j in range(101)]))
# Z2 = model2.predict(np.c_[xx.ravel(), yy.ravel()])
# Z2 = Z2.reshape(xx.shape)
# Z3 = model3.predict(np.c_[xx.ravel(), yy.ravel()])
# Z3 = Z3.reshape(xx.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [113]:
print(Z1)
print(sum(sum(Z1)))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]
0.0


In [114]:
print(np.matrix([float(instances[1]["y"][3][j]) for j in range(101)]))
print(sum([(instances[1]["y"][3][j]) for j in range(101)]))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]
1


Another model, a bit deeper

In [115]:
from keras.optimizers import SGD
loss_fun = ks.losses.BinaryFocalCrossentropy(alpha=0.99, gamma=3.0, from_logits=False)
model1_2 = Sequential()
model1_2.add(Dense(202, input_dim=101, activation='relu', kernel_initializer='he_uniform'))
model1_2.add(Dense(202, input_dim=101, activation='linear'))
model1_2.add(Dense(101, input_dim=101, activation='linear'))
model1_2.add(Dense(101, activation='sigmoid'))
# model2 = Sequential()
# model2.add(Dense(5, input_dim=101, activation='relu'))
# model2.add(Dense(1, activation='sigmoid'))
# model3 = Sequential()
# model3.add(Dense(20, input_dim=101, activation='relu'))
# model3.add(Dense(1, activation='sigmoid'))
# compile the models. Here we need to chose an optimiser. This one is called adam, it is used to
# determine how the training is performed. We do not care in this lecture how this is done.
# opt = SGD(learning_rate=0.01, momentum=0.9)
opt = ks.optimizers.Adam()
model1_2.compile(loss=loss_fun, optimizer=opt, metrics=['accuracy'])
# model2.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
# model3.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
#fit the models on the dataset
model1_2.fit(X, y, epochs=2, batch_size=5, verbose = True)
# model2.fit(X, y, epochs=30, batch_size=5, verbose = False)
# model3.fit(X, y, epochs=30, batch_size=5, verbose = False)
# evaluate the keras model
print('Model 1:')
_, accuracy1 = model1_2.evaluate(X, y)
# print('Model 2:')
# _, accuracy2 = model2.evaluate(X, y)
# print('Model 3:')
# _, accuracy3 = model3.evaluate(X, y)

Epoch 1/2
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 24s 1ms/step - accuracy: 0.0109 - loss: 0.0902
Epoch 2/2
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 27s 1ms/step - accuracy: 0.0102 - loss: 0.0097
Model 1:
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.0099 - loss: 0.0097


In [116]:
Z1 = model1_2.predict(np.matrix([instances[1]["dist"][1,j] if 1 != j else sqrt(100*100.0 + 100*100.0) for j in range(101)]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


In [117]:
print(Z1)
print(sum(sum(Z1)))

[[0.01473174 0.23655136 0.23133337 0.23703848 0.19692957 0.23066084
  0.2070946  0.21310243 0.21153921 0.21676    0.21180372 0.22176185
  0.2077828  0.2182491  0.23437664 0.24559239 0.2147649  0.2340131
  0.22993916 0.22052315 0.20247912 0.2130776  0.22978856 0.20116645
  0.21824174 0.2067146  0.20885311 0.22860284 0.2207869  0.23778221
  0.24484864 0.23377241 0.20598629 0.2273942  0.22131947 0.19265515
  0.22484556 0.23445925 0.21631151 0.23303863 0.22908975 0.24397266
  0.22070146 0.20892036 0.20932968 0.21041709 0.21176834 0.22196324
  0.23400433 0.22338833 0.22215113 0.21307416 0.22418606 0.20828068
  0.23031922 0.20774001 0.21964325 0.2263481  0.23927183 0.21295051
  0.22372581 0.23739396 0.22676735 0.21427941 0.23209684 0.22279291
  0.20588398 0.21470055 0.20530882 0.23602715 0.21132343 0.22000007
  0.22309065 0.2069359  0.2192657  0.22031341 0.19170344 0.21505207
  0.21961118 0.22255594 0.2265859  0.22561623 0.21189249 0.24195282
  0.22134562 0.22805391 0.22578688 0.23353975 0.2

In [118]:
print(np.matrix([float(instances[1]["y"][1][j]) for j in range(101)]))
print(sum([(instances[1]["y"][1][j]) for j in range(101)]))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]
1


In [119]:
import numpy as np

In [121]:
graph_data = np.array([[[instances[k]["nodes"][i][0], instances[k]["nodes"][i][1]] for i in range(101)] for k in range(100) for _ in range(101*100)])
print("next stage")
node_pairs_i = []
node_pairs_j = []
objective_vals = []
for k in range(100):
    for i in range(101):
        for j in range(101):
            if i != j:
                node_pairs_i.append([instances[k]["nodes"][i][0], instances[k]["nodes"][i][1]])
                node_pairs_j.append([instances[k]["nodes"][j][0], instances[k]["nodes"][j][1]])
                objective_vals.append(instances[k]["y"][i][j])
node_pairs_i = np.matrix(node_pairs_i)
node_pairs_j = np.matrix(node_pairs_j)
objective_vals = np.array(objective_vals)

next stage


In [122]:
from keras.models import Model
from keras.layers import concatenate, Input, Dense, Flatten
# define two sets of inputs
inputA = Input(shape=(101,2))
inputB = Input(shape=(2,))
inputC = Input(shape=(2,))
# the first branch operates on the first input
x_graph = Dense(101*101, activation="linear")(inputA)
x_graph = Dense(101, activation="relu")(x_graph)
x_graph = Dense(2, activation="linear")(x_graph)
x_graph = Flatten()(x_graph)
x_graph = Model(inputs=inputA, outputs=x_graph)
# the second branch opreates on the second input
x_node_i = Dense(2, activation="linear")(inputB)
x_node_i = Model(inputs=inputB, outputs=x_node_i)
# the third branch opreates on the third input
x_node_j = Dense(2, activation="linear")(inputC)
x_node_j = Model(inputs=inputC, outputs=x_node_j)
# combine the output of the two branches
combined = concatenate([x_graph.output, x_node_i.output, x_node_j.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(101, activation="linear")(combined)
z = Dense(2, activation="relu")(z)
z = Dense(2, activation="linear")(z)
z = Dense(1, activation="sigmoid")(z)
# our model will accept the inputs of the two branches and
# then output a single value
model1_3 = Model(inputs=[x_graph.output, x_node_i.output, x_node_j.output], outputs=z)

In [123]:
from keras.optimizers import SGD
loss_fun = ks.losses.BinaryFocalCrossentropy(alpha=0.99, gamma=3.0, from_logits=False)

In [130]:
# opt = SGD(learning_rate=0.01, momentum=0.9)
opt = ks.optimizers.Adam()
model1_3.compile(loss='binary_focal_crossentropy', optimizer=opt, metrics=['accuracy'])
model1_3.fit(x=[Flatten()(graph_data), node_pairs_i, node_pairs_j], y=objective_vals, epochs=2, batch_size=5, verbose = True)
print('Model 1_3:')
_, accuracy1 = model1_3.evaluate([Flatten()(graph_data), node_pairs_i, node_pairs_j], objective_vals)

Epoch 1/2
  3726/202000 ━━━━━━━━━━━━━━━━━━━━ 2:22 719us/step - accuracy: 0.9901 - loss: nan

KeyboardInterrupt: 

In [125]:
cnt = 0
for i in range(len(objective_vals)):
    if objective_vals[i] == 1:
        print(i)
        cnt += 1
    if cnt > 10:
        break

127
284
362
496
561
698
717
891
978
1014
1147


In [126]:
graph_ref = []
for i in range(len(graph_data[127].tolist())):
    graph_ref.append(graph_data[127][i].tolist()[0])
    graph_ref.append(graph_data[127][i].tolist()[1])
graph_ref = np.array([graph_ref])

In [127]:
Z1 = model1_3.predict(x=[graph_ref, node_pairs_i[127], node_pairs_j[127]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


In [128]:
print(Z1)

[[nan]]


In [55]:
node_pairs_i[234]

matrix([[32, 13]])

In [56]:
node_pairs_j[234]

matrix([[34, 16]])

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
from keras.models import Model
from keras.layers import concatenate, Input, Dense, Flatten

In [6]:


class KNNSimilarityLayer(Layer):
    def __init__(self, train_data, k=3, **kwargs):
        self.train_data = train_data
        self.k = k
        super(KNNSimilarityLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        super(KNNSimilarityLayer, self).build(input_shape)

    def call(self, inputs):
        # Expand dimensions to calculate distances
        inputs_expanded = tf.reshape(tf.expand_dims(inputs, 1), (101,2))
        t1 = tf.cast(tf.reshape(inputs_expanded, (1,101,2)), tf.float16)
        t2 = tf.cast(tf.reshape(inputs_expanded, (101,1,2)), tf.float16)
        
        # Calculate Euclidean distances
        distances = tf.norm(t1-t2, ord='euclidean', axis=2,)
        
        # Get the indices of the k nearest neighbors
        _, indices = tf.nn.top_k(-distances, k=self.k)

        # Create row indices (repeated for each column index in idx_tensor)
        row_indices = tf.repeat(tf.range(indices.shape[0]), indices.shape[1])

        # Flatten the column indices
        col_indices = tf.reshape(indices, [-1])

        # Combine row and column indices to form the full index tensor
        indices = tf.stack([row_indices, col_indices], axis=1)

        # Create updates tensor (all ones)
        updates = tf.ones(indices.shape[0], dtype=tf.int32)

        # Define the shape of the output tensor
        output_shape = [101, 101]

        # Scatter the updates into the output tensor
        prediction = tf.expand_dims(tf.scatter_nd(indices, updates, output_shape), 0)
        
        # # Gather the labels of the k nearest neighbors
        # knn_labels = tf.gather(self.train_labels, indices)
        
        # # Aggregate the labels (mode)
        # knn_labels = tf.cast(knn_labels, tf.float32)
        # prediction = tf.reduce_mean(knn_labels, axis=1)
        
        return prediction

In [60]:
from keras.layers import Reshape

In [68]:
inputs_expanded = tf.reshape(tf.expand_dims(unique_graph_data[0], 1), (101,2))
train_data_expanded = tf.expand_dims(unique_output_data[0], 0)[0]

In [69]:
train_data_expanded
print(tf.math.reduce_sum(train_data_expanded, 0))

tf.Tensor(
[0 0 1 1 0 0 2 2 2 1 2 1 2 2 2 0 0 2 2 0 1 3 1 1 2 1 1 0 1 1 1 1 1 1 0 1 0
 0 2 1 1 2 0 0 1 3 1 3 1 1 1 0 1 0 2 0 1 0 1 1 1 1 1 2 1 0 0 1 2 1 0 1 1 2
 2 0 1 1 0 2 0 1 2 0 1 1 1 1 1 0 1 3 1 1 0 0 1 1 1 1 0], shape=(101,), dtype=int32)


In [104]:
idx_tensor = tf.constant(np.array([[1 , 2], [0, 3]]))
# Create row indices (repeated for each column index in idx_tensor)
row_indices = tf.repeat(tf.range(idx_tensor.shape[0]), idx_tensor.shape[1])

# Flatten the column indices
col_indices = tf.reshape(idx_tensor, [-1])

# Combine row and column indices to form the full index tensor
indices = tf.stack([row_indices, col_indices], axis=1)

# Create updates tensor (all ones)
updates = tf.ones(indices.shape[0], dtype=tf.int32)

# Define the shape of the output tensor
output_shape = [2, 4]

# Scatter the updates into the output tensor
output_tensor = tf.scatter_nd(indices, updates, output_shape)

print(output_tensor)

tf.Tensor(
[[0 1 1 0]
 [1 0 0 1]], shape=(2, 4), dtype=int32)


In [70]:
t1 = tf.cast(tf.reshape(inputs_expanded, (1,101,2)), tf.float16)
t2 = tf.cast(tf.reshape(inputs_expanded, (101,1,2)), tf.float16)
result = tf.norm(t1-t2, ord='euclidean', axis=2,)

In [81]:
# print(result)
_, indices = tf.nn.top_k(-result, k=3)
print(indices)

tf.Tensor(
[[  0   1  14]
 [  1   0  14]
 [  2  35   4]
 [  3   8  23]
 [  4  92  99]
 [  5  73  62]
 [  6  46  49]
 [  7  67  43]
 [  8   3  23]
 [  9  68  22]
 [ 10  97  75]
 [ 11  45  95]
 [ 12  58  85]
 [ 13  72  78]
 [ 14  91  70]
 [ 15  47  55]
 [ 16  74  84]
 [ 17  82  19]
 [ 18  50  56]
 [ 19  17  82]
 [ 20  81  74]
 [ 21  38  57]
 [ 22  68   9]
 [ 23   8   3]
 [ 24  33  83]
 [ 25  52  31]
 [ 26  69  95]
 [ 27  56   7]
 [ 28  85  74]
 [ 29  64  77]
 [ 30  98  39]
 [ 31  90  25]
 [ 32  59  71]
 [ 33  24  83]
 [ 34  21  38]
 [ 35   2  64]
 [ 36  11  19]
 [ 37  38  21]
 [ 38  21  57]
 [ 39  62  30]
 [ 40  86  52]
 [ 41  60  89]
 [ 42   9  68]
 [ 43   7  60]
 [ 44  54  78]
 [ 45  73  95]
 [ 46   6 100]
 [ 47  77  55]
 [ 48  61  82]
 [ 49   6 100]
 [ 50  18  56]
 [ 51  23   8]
 [ 52  25  31]
 [ 53  82  17]
 [ 54  44  63]
 [ 55  47  77]
 [ 56  18  27]
 [ 57  21  38]
 [ 58  12  85]
 [ 59  32  93]
 [ 60  41  43]
 [ 61  48  82]
 [ 62  39  34]
 [ 63  76  54]
 [ 64  29  77]
 [ 65  63  76]

In [88]:
prediction = tf.Variable(tf.zeros((101,101)))
for i in range(101):
    for j in indices[i]:
        prediction[i][j].assign(1)
print(prediction)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

In [7]:
unique_graph_data = np.array([[[instances[k]["nodes"][i][0], instances[k]["nodes"][i][1]] for i in range(101)] for k in range(100)])

In [15]:
unique_graph_data[2]

array([[  0,   0],
       [ 98,  24],
       [  5,  80],
       [  7,  65],
       [ 52,  63],
       [ 14,  64],
       [ 63,  55],
       [ 26,  13],
       [ 40,  84],
       [ 34,  85],
       [ 74,  97],
       [ 31,  24],
       [ 90,   5],
       [ 38,  76],
       [ 22,  13],
       [  8,  65],
       [ 32,  41],
       [ 45,  32],
       [ 78,  31],
       [ 51,  92],
       [ 65,  29],
       [ 95,  79],
       [ 74,  26],
       [ 72,  14],
       [ 52,   4],
       [ 64,  78],
       [ 39,  65],
       [ 71,  32],
       [  9,  68],
       [  5,  97],
       [ 24,  91],
       [  2,  57],
       [ 37,  79],
       [ 75,  16],
       [ 28,  64],
       [ 82,  75],
       [  1,  29],
       [ 51,  64],
       [ 80,  84],
       [ 30,  56],
       [ 30,  90],
       [ 33,  13],
       [ 47,  39],
       [  5,  40],
       [ 68,  36],
       [  8,  42],
       [ 25,  30],
       [ 89,  35],
       [ 21,  10],
       [ 78,  36],
       [ 18,  69],
       [ 26,  66],
       [ 56,

In [8]:
unique_output_data = []
for k in range(100):
    unique_output_data.append([])
    for l in y[101*k:101*(k+1)]:
        unique_output_data[k].append(l.tolist()[0])
unique_output_data = np.array(unique_output_data)

In [9]:
knn_layer = KNNSimilarityLayer( np.float32(unique_graph_data), k=3)


In [62]:
from keras.layers import Reshape

In [25]:
inputs = Input(shape=(101,2))
# knn_output = knn_layer(inputs)
dense_output = Dense(32, activation='relu')(inputs)
dense_output = Dense(16, activation='relu')(dense_output)
dense_output = Dense(2, activation='relu')(dense_output)
# outputs = Dense(101, activation='sigmoid')(dense_output)
# knn_output = knn_layer(inputs)
# outputs = Reshape((1, 101,101)) (outputs)
knn_l = knn_layer(dense_output)
dense_output = Dense(32, activation='relu')(knn_l)
outputs = Dense(101, activation='sigmoid')(dense_output)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_focal_crossentropy', metrics=['accuracy'])

In [26]:
tf.config.experimental_run_functions_eagerly(True)

In [27]:
model.fit(np.float32(unique_graph_data), np.float32(unique_output_data), epochs=20, batch_size=1)

Epoch 1/20
  3/100 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.0132 - loss: 0.1752  

c:\Users\jotape\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\jotape\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.0110 - loss: 0.1429
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.0124 - loss: 0.0277
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.0097 - loss: 0.0177
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.0119 - loss: 0.0173
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.0124 - loss: 0.0171
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.0110 - loss: 0.0171
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.0129 - loss: 0.0170
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.0148 - loss: 0.0170
Epoch 9/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.0153 - loss: 0.0170
Epoch 10/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.0155 - loss: 0.0169
Epoch 11/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.0156 - loss: 0.0169
Epoch 12/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/ste

In [30]:
Z1 = model.predict(np.float32([unique_graph_data[2]]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [33]:
print(Z1[0])

[[0.04841506 0.15015088 0.13757032 ... 0.15883666 0.14912005 0.13748477]
 [0.04841506 0.15015088 0.13757032 ... 0.15883666 0.14912005 0.13748477]
 [0.02741079 0.13931516 0.15677123 ... 0.11270387 0.16925497 0.11983629]
 ...
 [0.04841506 0.15015088 0.13757032 ... 0.15883666 0.14912005 0.13748477]
 [0.04841506 0.15015088 0.13757032 ... 0.15883666 0.14912005 0.13748477]
 [0.04841506 0.15015088 0.13757032 ... 0.15883666 0.14912005 0.13748477]]


In [32]:
np.float32(unique_output_data[2])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [40]:
max_z = -1
loc_max = (-1,-1)
for i in range(101):
    for j in range(101):
        print(Z1[0][i][j], unique_output_data[2][i][j], i, j)
        if Z1[0][i][j] > max_z:
            max_z = Z1[0][i][j]
            loc_max = (i,j)


0.048415065 0 0 0
0.15015088 0 0 1
0.13757032 0 0 2
0.15058713 0 0 3
0.16793248 0 0 4
0.17339277 0 0 5
0.15798728 0 0 6
0.14808802 0 0 7
0.14341398 0 0 8
0.15139583 0 0 9
0.16270068 0 0 10
0.1478354 0 0 11
0.13566975 0 0 12
0.14300357 0 0 13
0.15525216 0 0 14
0.1397806 0 0 15
0.15332273 0 0 16
0.15700038 0 0 17
0.16471793 0 0 18
0.15010263 0 0 19
0.15655258 0 0 20
0.15318488 0 0 21
0.16376628 0 0 22
0.17144418 0 0 23
0.15089743 0 0 24
0.14256571 0 0 25
0.14655547 0 0 26
0.16285998 0 0 27
0.14810354 0 0 28
0.14382489 0 0 29
0.13414155 0 0 30
0.15024199 0 0 31
0.15424964 0 0 32
0.15176545 0 0 33
0.16182093 0 0 34
0.15711966 0 0 35
0.15537179 0 0 36
0.15717629 0 0 37
0.16496553 0 0 38
0.14566931 0 0 39
0.14526483 0 0 40
0.1536305 0 0 41
0.15323533 0 0 42
0.14782685 0 0 43
0.13772777 0 0 44
0.15134777 0 0 45
0.1330752 0 0 46
0.15483129 0 0 47
0.17505126 0 0 48
0.16806018 0 0 49
0.13491149 0 0 50
0.13817212 0 0 51
0.13203357 0 0 52
0.15928075 0 0 53
0.15678075 0 0 54
0.16217113 0 0 55
0.160

In [41]:
print(max_z)
print(loc_max)

0.2458665
(45, 71)
